In [3]:
import configparser
import psycopg2
import boto3
import json
from tabulate import tabulate
import pandas as pd

In [4]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")

DB_NAME                 = config.get("CLUSTER","DB_NAME")
DB_USER            = config.get("CLUSTER","DB_USER")
DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DB_PORT               = config.get("CLUSTER","DB_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

SONG_DATA = config.get('S3','SONGS_DATA')
LOG_DATA = config.get('S3','LOG_DATA')
BUCKET = config.get('S3','BUCKET')

NoOptionError: No option 'song_data' in section: 'S3'

In [ ]:
print(type(BUCKET))

In [ ]:
ec2 = boto3.resource('ec2', region_name='us-west-2', aws_access_key_id=KEY, aws_secret_access_key=SECRET)
s3 = boto3.resource('s3', region_name='us-west-2', aws_access_key_id=KEY, aws_secret_access_key=SECRET)
iam = boto3.client('iam', region_name='us-west-2', aws_access_key_id=KEY, aws_secret_access_key=SECRET)
redshift = boto3.client('redshift', region_name='us-west-2', aws_access_key_id=KEY, aws_secret_access_key=SECRET)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [ ]:
log_data_bucket = s3.Bucket(BUCKET)

first_object = None

for my_object in log_data_bucket.objects.filter(Prefix='log_data').limit(2):
    if my_object.key.endswith('.json'):
        first_object = my_object
        print(my_object)

s3_client = boto3.client('s3', region_name='us-west-2', aws_access_key_id=KEY, aws_secret_access_key=SECRET)
json_obj = s3_client.get_object(Bucket=BUCKET, Key=first_object.key)
json_data = json_obj["Body"].read().decode('utf-8')
json_df = pd.read_json(json_data, lines=True)
print(json_df.info())
# print(tabulate(json_df.head(10), headers='keys', tablefmt='psql'))

In [ ]:
song_data_bucket = s3.Bucket(BUCKET)

first_object = None

for my_object in song_data_bucket.objects.filter(Prefix='song-data').limit(4):
    if my_object.key.endswith('.json'):
        first_object = my_object
        print(my_object)

s3_client = boto3.client('s3', region_name='us-west-2', aws_access_key_id=KEY, aws_secret_access_key=SECRET)
json_obj = s3_client.get_object(Bucket=BUCKET, Key=first_object.key)
json_data = json_obj["Body"].read().decode('utf-8')
json_df = pd.read_json(json_data, lines=True)
print(json_df.head())

In [ ]:
# song_data_bucket = s3.Bucket(BUCKET)

# first_object = None

# for my_object in song_data_bucket.objects.all():
#     if my_object.key.endswith('log_json_path.json'):
#         first_object = my_object
#         print(my_object)
#         break
# json_obj = s3_client.get_object(Bucket=BUCKET, Key=first_object.key)
# json_data = json_obj["Body"].read().decode('utf-8')
# json_dump = json.dumps(json_data, indent=4, separators=(',', ':'))
# json_df = pd.read_json(json_dump, lines=True)
# print(json_df.head())


In [ ]:
# TODO: Create the IAM role
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description="Allows Redshift clusters to call AWS services on your behalf",
        AssumeRolePolicyDocument= json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
                            'Effect': 'Allow',
                            'Principal': {'Service': 'redshift.amazonaws.com'}}],
            'Version': '2012-10-17'}
        )
    )
    iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                        PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                        )['ResponseMetadata']['HTTPStatusCode']
    

except Exception as e:
    print(e)

In [ ]:
# Get ARN role
print("Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
print(roleArn)

In [ ]:
# Create a RedShift cluster
try:
    response = redshift.create_cluster(        
        # TODO: add parameters for hardware
        ClusterType = DWH_CLUSTER_TYPE,
        NodeType = DWH_NODE_TYPE,
        NumberOfNodes = int(DWH_NUM_NODES),

        # TODO: add parameters for identifiers & credentials
        DBName = DB_NAME,
        ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
        MasterUsername = DB_USER,
        MasterUserPassword = DB_PASSWORD,
        
        # TODO: add parameter for role (to allow s3 access)
        IamRoles = [roleArn]
    )
except Exception as e:
    print(e)

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', None)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", roleArn)

In [ ]:
# Open incoming TCP port to access cluster endpoint
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName= defaultSg.group_name,  # TODO: fill out
        CidrIp='0.0.0.0/0',  # TODO: fill out
        IpProtocol='TCP',  # TODO: fill out
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DB_USER, DB_PASSWORD, DWH_ENDPOINT, DB_PORT,DB_NAME)
print(conn_string)

In [ ]:
%sql $conn_string

In [ ]:
%%sql
SELECT *
FROM staging_events LIMIT 3;